## Hotdog or Not HotDog

Welcome to this SageMaker Notebook! This is an entirely managed notebook service that you can use to create and edit machine learning models. We will be using it today to create a binary image classification model using the Apache MXNet deep learning framework. We will then learn how to delpoy this model onto our DeepLens device.

In this notebook we will be to using MXNet's Gluon interface, to download and edit a pre-trained ImageNet model and transform it into binary classifier, which we can use to differentiate between hot dogs and not hot dogs.

### Setup

Before we start, make sure the kernel in the the notebook is set to the correct one, `condamxnet3.6` which has all the dependencies we will need for this tutorial already installed.

First we'll start by importing a bunch of packages into the notebook that you'll need later and installing any required packages that are missing into our notebook kernel.

In [1]:
%%bash
conda install scikit-image

Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /home/ec2-user/anaconda3/envs/mxnet_p36:

The following packages will be UPDATED:

    scikit-image: 0.13.0-py36had3c07a_1 --> 0.13.1-py36ha4a0841_0

The following packages will be DOWNGRADED:

    networkx:     2.0-py36h7e96fb8_0    --> 1.11-py36hfb3574a_0  

Proceed ([y]/n)? 


In [2]:
from __future__ import print_function
import logging
logging.basicConfig(level=logging.INFO)
import os
import time
from collections import OrderedDict
import skimage.io as io
import numpy as np

import mxnet as mx

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/matplotlib/colors.py:680: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  not cbook.is_string_like(colors[0]):
/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


## Model

The model we will be downloading and editing is [SqueezeNet](https://arxiv.org/abs/1602.07360), an extremely efficient image classification model that achived 2012 State of the Art accuracy on the popular [ImageNet](http://www.image-net.org/challenges/LSVRC/), image classification challenge. SqueezeNet is just a convolutional neural network, with an architecture chosen to have a small number of parameters and to require a minimal amount of computation. It's especially popular for folks that need to run CNNs on low-powered devices like cell phones and other internet-of-things devices, such as DeepLens. The MXNet Deep Learning framework offers squeezenet v1.0 and v1.1 that are pretrained on ImageNet through it's model Zoo.

## Pulling the pre-trained model
The MXNet model zoo  gives us convenient access to a number of popular models,
both their architectures and their pretrained parameters.
Let's download SqueezeNet right now with just a few lines of code.

In [3]:
from mxnet.gluon import nn
from mxnet.gluon.model_zoo import vision as models

# get pretrained squeezenet
net = models.squeezenet1_1(pretrained=True, prefix='deep_dog_')
# hot dog happens to be a class in imagenet.
# we can reuse the weight for that class for better performance
# here's the index for that class for later use
imagenet_hotdog_index = 713

### DeepDog Net

In vision networks its common that the first set of layers learns the task of recognizing edges, curves and other important visual features of the input image. We call this feature extraction, and once the abstract features are extracted we can leverage a much simpler model to classify images using these features.

We will use the feature extractor from the pretrained squeezenet (every layer except the last one) to build our own classifier for hotdogs. Conveniently, the MXNet model zoo handles the decaptiation for us. All we have to do is specify the number out of output classes in our new task, which we do via the keyword argument `classes=2`.

In [4]:
deep_dog_net = models.squeezenet1_1(prefix='deep_dog_', classes=2)
deep_dog_net.collect_params().initialize()
deep_dog_net.features = net.features

# Lets take a look at what this network looks like
print(deep_dog_net)

SqueezeNet(
  (features): HybridSequential(
    (0): Conv2D(64, kernel_size=(3, 3), stride=(2, 2))
    (1): Activation(relu)
    (2): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=True)
    (3): HybridSequential(
      (0): HybridSequential(
        (0): Conv2D(16, kernel_size=(1, 1), stride=(1, 1))
        (1): Activation(relu)
      )
      (1): HybridConcurrent(
        (0): HybridSequential(
          (0): Conv2D(64, kernel_size=(1, 1), stride=(1, 1))
          (1): Activation(relu)
        )
        (1): HybridSequential(
          (0): Conv2D(64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): Activation(relu)
        )
      )
    )
    (4): HybridSequential(
      (0): HybridSequential(
        (0): Conv2D(16, kernel_size=(1, 1), stride=(1, 1))
        (1): Activation(relu)
      )
      (1): HybridConcurrent(
        (0): HybridSequential(
          (0): Conv2D(64, kernel_size=(1, 1), stride=(1, 1))
          (1): Activation(relu)
        )


The network can already be used for prediction. However, since it hasn't been finetuned yet so the network performance could not be optimal.

Let's test it out by defining a prediction function to feed a local image into the network and get the predicted output

In [5]:
from skimage.color import rgba2rgb

def classify_hotdog(net, url):
    I = io.imread(url)
    if I.shape[2] == 4:
        I = rgba2rgb(I)
    image = mx.nd.array(I).astype(np.uint8)
    image = mx.image.resize_short(image, 256)
    image, _ = mx.image.center_crop(image, (224, 224))
    image = mx.image.color_normalize(image.astype(np.float32)/255,
                                     mean=mx.nd.array([0.485, 0.456, 0.406]),
                                     std=mx.nd.array([0.229, 0.224, 0.225]))
    image = mx.nd.transpose(image.astype('float32'), (2,1,0))
    image = mx.nd.expand_dims(image, axis=0)
    out = mx.nd.SoftmaxActivation(net(image))
    print('Probabilities are: '+str(out[0].asnumpy()))
    result = np.argmax(out.asnumpy())
    outstring = ['Not hotdog!', 'Hotdog!']
    print(outstring[result])

Now lets download a hot dog image and an image of another object to our local directory to test this model on

In [6]:
%%bash
wget http://www.wienerschnitzel.com/wp-content/uploads/2014/10/hotdog_mustard-main.jpg
wget https://www.what-dog.net/Images/faces2/scroll001.jpg

--2017-11-24 08:32:34--  http://www.wienerschnitzel.com/wp-content/uploads/2014/10/hotdog_mustard-main.jpg
Resolving www.wienerschnitzel.com (www.wienerschnitzel.com)... 104.198.109.247
Connecting to www.wienerschnitzel.com (www.wienerschnitzel.com)|104.198.109.247|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22917 (22K) [image/jpeg]
Saving to: ‘hotdog_mustard-main.jpg.1’

     0K .......... .......... ..                              100%  358K=0.06s

2017-11-24 08:32:34 (358 KB/s) - ‘hotdog_mustard-main.jpg.1’ saved [22917/22917]

--2017-11-24 08:32:34--  https://www.what-dog.net/Images/faces2/scroll001.jpg
Resolving www.what-dog.net (www.what-dog.net)... 191.237.47.20
Connecting to www.what-dog.net (www.what-dog.net)|191.237.47.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48316 (47K) [image/jpeg]
Saving to: ‘scroll001.jpg’

     0K .......... .......... .......... .......... .......   100% 8.58M=0.005s

2017-11-24 08:32:34 (

In [7]:
# To make the defined network run quickly we usually hybridize it first. 
# This also allows us to serialize and export our model
deep_dog_net.hybridize()

# Let's run the classification on our tow downloaded images to see what our model comes up with
classify_hotdog(deep_dog_net, './hotdog_mustard-main.jpg') # check for hotdog
classify_hotdog(deep_dog_net, './scroll001.jpg') # check for not-hotdog

Probabilities are: [ 0.66635329  0.33364674]
Not hotdog!
Probabilities are: [ 0.48589313  0.51410687]
Hotdog!


In [8]:
deep_dog_net.export('hotdog_or_not_model')

The predictions are a bit off so we can download a set of new parameters for the model that we have pre-optimized through a "fine tuning" process, where we retrained the model with images of hotdogs and not hotdogs. We can then apply these new parameters to our model to make it even more accurate.

In [9]:
from mxnet.test_utils import download

download('https://apache-mxnet.s3-accelerate.amazonaws.com/gluon/models/deep-dog-5a342a6f.params',
         overwrite=True)
deep_dog_net.load_params('deep-dog-5a342a6f.params', mx.cpu())
deep_dog_net.hybridize()
classify_hotdog(deep_dog_net, './hotdog_mustard-main.jpg')
classify_hotdog(deep_dog_net, './scroll001.jpg')


INFO:root:downloaded https://apache-mxnet.s3-accelerate.amazonaws.com/gluon/models/deep-dog-5a342a6f.params into deep-dog-5a342a6f.params successfully


Probabilities are: [ 0.37135434  0.62864566]
Hotdog!
Probabilities are: [ 0.99881637  0.00118361]
Not hotdog!


The predictions seem reasonable, so we can export this as a serialized model to our local dirctory. This is a simple one line command, which produces a set of two files: a json file holding the network architecture, and a params file holding the parameters the network learned.

In [10]:
deep_dog_net.export('hotdog_or_not_model_v2')

Now let's push this serialized model to S3, where we can then optimize it for our DeepLense device and then push it down onto our device for inference.

In [14]:
import boto3
import re

assumed_role = boto3.client('sts').get_caller_identity()['Arn']
s3_access_role = re.sub(r'^(.+)sts::(\d+):assumed-role/(.+?)/.*$', r'\1iam::\2:role/\3', assumed_role)
print(s3_access_role)
s3 = boto3.resource('s3')
bucket= 'your s3 bucket name here' 

json = open('hotdog_or_not_model-symbol.json', 'rb')
params = open('hotdog_or_not_model-0000.params', 'rb')
s3.Bucket(bucket).put_object(Key='test/hotdog_or_not_model-symbol.json', Body=json)
s3.Bucket(bucket).put_object(Key='test/hotdog_or_not_model-0000.params', Body=params)

INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): sts.amazonaws.com
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com


arn:aws:iam::622803848910:role/SageMaker_role_IM


s3.Object(bucket_name='sagemaker-test1', key='hotdog_or_not_model-0000.params')